## **Data Quality**
 - Duplicate Records:
   Identify and quantify duplicate entries (e.g., based on FORMATTED_ADDRESS).
 - Inconsistent or Invalid Data:
   Check for unusual values in BEDS, BATH (e.g., negative values, unrealistic ranges).
 - Validate geospatial coordinates (LATITUDE, LONGITUDE) for out-of-bound values.
 - Data Completeness:
   Assess completeness of address-related fields (ADDRESS, LOCALITY, etc.).


In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np

In [5]:
house_data = pd.read_csv("C:/Users/1999i/Documents/data mange/Lab Lectures/Project/Data/NY-House-Dataset.csv")
q = house_data.shape
print(f"Original data shape {q}")
# Extract the names before the comma
house_data['State_City'] = house_data['STATE'].str.extract(r'^([\w\s]+),')
# Extracting new york county regions
NY_county = house_data[house_data['State_City'].isin(['New York', 'Manhattan', 'Roosevelt Island', 'New York City'])]
qq = NY_county.shape
print(f"Properties located only in New York County data shape {qq}")

Original data shape (4801, 17)
Properties located only in New York County data shape (1295, 18)


In [33]:
NY_county = NY_county[['BROKERTITLE', 'TYPE', 'PRICE', 'BEDS','BATH','PROPERTYSQFT','MAIN_ADDRESS','STATE', 'State_City','LATITUDE','LONGITUDE']]
NY_county.head(3)

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,MAIN_ADDRESS,STATE,State_City,LATITUDE,LONGITUDE
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.0,1400.0,"2 E 55th St Unit 803New York, NY 10022","New York, NY 10022",New York,40.761255,-73.974483
1,Brokered by Serhant,Condo for sale,195000000,7,10.0,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",New York,40.766393,-73.980991
3,Brokered by COMPASS,Condo for sale,69000,3,1.0,445.0,"2 E 55th St Unit 908W33Manhattan, NY 10022","Manhattan, NY 10022",Manhattan,40.761398,-73.974613


In [14]:
duplicates = NY_county[NY_county.duplicated(subset=['BROKERTITLE', 'MAIN_ADDRESS'])]
w = duplicates.shape
print(f"Duplicate rows shape: {w}")
NY_county_cleaned = NY_county.drop_duplicates(subset=['BROKERTITLE', 'MAIN_ADDRESS'])
ww = NY_county_cleaned.shape
print(f"After removing Duplicate rows shape: {ww}")

Duplicate rows shape: (64, 11)
After removing Duplicate rows shape: (1231, 11)


In [16]:
# Validate geospatial coordinates (LATITUDE, LONGITUDE) for out-of-bound values.
# New York City bounds
nyc_bounds = {
    'lat_min': 40.4774,
    'lat_max': 40.9176,
    'lon_min': -74.2591,
    'lon_max': -73.7002
}

# Filter data valid coords
dt_county = NY_county_cleaned[
    (NY_county_cleaned['LATITUDE'] >= nyc_bounds['lat_min']) &
    (NY_county_cleaned['LATITUDE'] <= nyc_bounds['lat_max']) &
    (NY_county_cleaned['LONGITUDE'] >= nyc_bounds['lon_min']) &
    (NY_county_cleaned['LONGITUDE'] <= nyc_bounds['lon_max'])
]

e = dt_county.shape
print(f"Valid coordinates shape: {e}")

invalid_coords = NY_county_cleaned[~NY_county_cleaned.index.isin(dt_county.index)]
ee = invalid_coords.shape
print(f"Invalid coordinates shape: {ee}")

Valid coordinates shape: (1231, 11)
Invalid coordinates shape: (0, 11)


In [30]:
dt_county.TYPE.unique()

array(['Condo for sale', 'Townhouse for sale', 'Co-op for sale',
       'For sale', 'House for sale', 'Pending', 'Foreclosure',
       'Multi-family home for sale', 'Land for sale', 'Condop for sale',
       'Contingent'], dtype=object)

In [32]:
dt_county.BROKERTITLE.unique()

array(['Brokered by Douglas Elliman  -111 Fifth Ave',
       'Brokered by Serhant', 'Brokered by COMPASS',
       "Brokered by Sotheby's International Realty - East Side Manhattan Brokerage",
       'Brokered by Douglas Elliman - 575 Madison Ave',
       'Brokered by Corcoran Chelsea/Flatiron',
       'Brokered by Brown Harris Stevens',
       'Brokered by Brown Harris Stevens - 445 Park Avenue',
       'Brokered by Brown Harris Stevens - 1926 Broadway',
       'Brokered by Engel & Volkers Brownstone Brooklyn',
       'Built by Toll Brothers', 'Brokered by Avenues - Headquarters',
       'Brokered by Corcoran East Side', 'Brokered by Exp Realty',
       'Brokered by Island Advantage Rlty, Llc',
       'Brokered by Garfield, Leslie J. & Co., Inc.',
       'Brokered by Corcoran West Side',
       'Brokered by All American Realty Management',
       'Brokered by Byson Real Estate Company - Byson Real Estate Company',
       "Brokered by CHRISTIE'S INT.REAL ESTATE GROUP",
       'Brokered 

## **Data Profiling**
- Missing values
- filtering whole numbers for beds and bath
- variable overview : summary statistics
- Data Distribution Analysis: Histograms: Plot histograms for numerical variables (PRICE, BEDS, BATH, scores) to visualize their distributions.
- Pie Charts: Proportions of house types (TYPE) or distribution of states (STATE).
- Spatial Analysis: Heatmap: Use heatmap overlays to show density of houses or amenities.

In [34]:
print(f"Missing data in New York County:")
dt_county.isna().sum()

Missing data in New York County:


BROKERTITLE     0
TYPE            0
PRICE           0
BEDS            0
BATH            0
PROPERTYSQFT    0
MAIN_ADDRESS    0
STATE           0
State_City      0
LATITUDE        0
LONGITUDE       0
dtype: int64

In [35]:
# variable overview : summary statistics
dt_county.describe()

,PRICE,BEDS,BATH,PROPERTYSQFT,LATITUDE,LONGITUDE
count,1.231000e+03,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000
mean,6.029277e+06,2.968318,2.459236,2497.490908,40.766458,-73.973598
std,6.173164e+07,2.260614,1.932233,2157.991161,0.033652,0.022997
min,2.494000e+03,1.000000,0.000000,230.000000,40.518484,-74.224418
25%,6.980000e+05,2.000000,1.000000,1431.000000,40.743561,-73.987420
50%,1.495000e+06,3.000000,2.000000,2184.207862,40.768047,-73.972208
75%,4.172500e+06,3.000000,3.000000,2184.207862,40.783670,-73.958624
max,2.147484e+09,32.000000,20.000000,24000.000000,40.880930,-73.825917


In [40]:
# decimals in beds and baths
dt_county['beds_has_decimal'] = dt_county['BEDS'] % 1 != 0  # TRUE if ther is decimal
dt_county['bath_has_decimal'] = dt_county['BATH'] % 1 != 0  
beds_decimal_count = dt_county['beds_has_decimal'].sum()
bath_decimal_count = dt_county['bath_has_decimal'].sum()

print(f"Rows with decimal values in 'beds': {beds_decimal_count}")
print(f"Rows with decimal values in 'bath': {bath_decimal_count}")

Rows with decimal values in 'beds': 0
Rows with decimal values in 'bath': 49


In [55]:
# filtering only whole numbers.
dt_county_ =  dt_county[dt_county['bath_has_decimal'] == False]
z = dt_county_.shape
print(f"Data shape filtering only whole numbers for beds and bath: {z}")

Data shape filtering only whole numbers for beds and bath: (1182, 13)


In [56]:
# variable overview : summary statistics
dt_county_.describe()

,PRICE,BEDS,BATH,PROPERTYSQFT,LATITUDE,LONGITUDE
count,1.182000e+03,1182.000000,1182.000000,1182.000000,1182.000000,1182.000000
mean,5.636089e+06,2.900169,2.462775,2357.622515,40.766465,-73.973607
std,6.289851e+07,2.259278,1.971830,1864.468974,0.033680,0.023013
min,2.494000e+03,1.000000,0.000000,230.000000,40.518484,-74.224418
25%,6.792500e+05,2.000000,1.000000,1375.250000,40.743942,-73.987286
50%,1.412500e+06,3.000000,2.000000,2184.207862,40.767687,-73.972395
75%,3.898750e+06,3.000000,3.000000,2184.207862,40.783662,-73.958623
max,2.147484e+09,32.000000,20.000000,24000.000000,40.880930,-73.825917


## **Data Integration/ Data Enrichment**

In [58]:
walkscore = pd.read_csv("C:/Users/1999i/Documents/data mange/Lab Lectures/Project/Data/walkscore.csv")
q = walkscore.shape
print(f"walkscore data shape: {q}")

walkscore data shape: (93, 5)


In [59]:
walkscore.head(3)

,Location,Bike Score,Transit Score,Walk Score,Neighborhood
0,"Midtown East, New York, 10022",82,100.0,100,"New York, NY 10022"
1,"Hell's Kitchen, New York, 10019",91,100.0,93,"New York, NY 10019"
2,"Upper East Side, Manhattan, 10065",83,100.0,99,"New York, NY 10065"


In [72]:
walkscore.Location.unique()

array(['Midtown East, New York, 10022', "Hell's Kitchen, New York, 10019",
       'Upper East Side, Manhattan, 10065', 'Manhattan, New York, 10027',
       'West Village, New York, 10014', 'SoHo, New York, 10013',
       'Two Bridges, Manhattan, 10002',
       'Upper East Side, Manhattan, 10075', 'Manhattan, New York, 10128',
       'SoHo, New York, 10012', 'Manhattan, New York, 10025',
       'Manhattan, New York, 10023', 'East Harlem, New York, 10029',
       'West Harlem, Manhattan, 10031',
       'Upper East Side, Manhattan, 10021',
       'Washington Heights, New York, 10032', 'Tribeca, New York, 10007',
       'Upper West Side, Manhattan, 10024',
       'Murray Hill, New York, 10016', "Hell's Kitchen, New York, 10036",
       'Manhattan, New York, 10028', 'Midtown East, New York, 10017',
       'New York, New York, 10033', 'Charleston, New York, 10309',
       'Chelsea, New York, 10001', 'Greenwich Village, New York, 10003',
       'New York, New York, 10069', 'Kingsbridge, New Y

In [60]:
# left merge dataset (Integration)
NY_walkscore = pd.merge(dt_county_, walkscore, how='left', left_on='STATE', right_on='Neighborhood')
print(f"Missing Values: ")
NY_walkscore.isna().sum()
rows_with_na = NY_walkscore[NY_walkscore.isna().any(axis=1)] 

Missing Values: 


BROKERTITLE         0
TYPE                0
PRICE               0
BEDS                0
BATH                0
PROPERTYSQFT        0
MAIN_ADDRESS        0
STATE               0
State_City          0
LATITUDE            0
LONGITUDE           0
beds_has_decimal    0
bath_has_decimal    0
Location            0
Bike Score          0
Transit Score       6
Walk Score          0
Neighborhood        0
dtype: int64

In [61]:
NY_walkscore.shape

(1182, 18)

In [65]:
NY_walkscore.head(3)

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,MAIN_ADDRESS,STATE,State_City,LATITUDE,LONGITUDE,beds_has_decimal,bath_has_decimal,Location,Bike Score,Transit Score,Walk Score,Neighborhood
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.0,1400.0,"2 E 55th St Unit 803New York, NY 10022","New York, NY 10022",New York,40.761255,-73.974483,False,False,"Midtown East, New York, 10022",82,100.0,100,"New York, NY 10022"
1,Brokered by Serhant,Condo for sale,195000000,7,10.0,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",New York,40.766393,-73.980991,False,False,"Hell's Kitchen, New York, 10019",91,100.0,93,"New York, NY 10019"
2,Brokered by COMPASS,Condo for sale,69000,3,1.0,445.0,"2 E 55th St Unit 908W33Manhattan, NY 10022","Manhattan, NY 10022",Manhattan,40.761398,-73.974613,False,False,"Midtown East, New York, 10022",82,100.0,100,"Manhattan, NY 10022"


In [66]:
# Data extracted from OSM
# public transport
public_trans = pd.read_csv("C:/Users/1999i/Documents/data mange/Lab Lectures/Project/new_york_amenities-public_transport.csv")

# hospitals
hospitals = pd.read_csv("C:/Users/1999i/Documents/data mange/Lab Lectures/Project/new_york_amenities-hospitals.csv")

# schools
schools = pd.read_csv("C:/Users/1999i/Documents/data mange/Lab Lectures/Project/new_york_amenities-schools.csv")

In [67]:
# Convert to geo-dataframes
# NY_walkscore
NY_gdf = gpd.GeoDataFrame(NY_walkscore, crs='EPSG:4326',
    geometry=gpd.points_from_xy(NY_walkscore.LONGITUDE, NY_walkscore.LATITUDE))

# public transport
trans_gdf = gpd.GeoDataFrame(public_trans,crs='EPSG:4326',
    geometry=gpd.points_from_xy(public_trans.lon, public_trans.lat))

# hospitals
hospitals_gdf = gpd.GeoDataFrame(hospitals,crs='EPSG:4326',
    geometry=gpd.points_from_xy(hospitals.lon, hospitals.lat))

# schools
schools_gdf = gpd.GeoDataFrame(schools,crs='EPSG:4326',
    geometry=gpd.points_from_xy(schools.lon, schools.lat))


In [68]:
from shapely.ops import nearest_points


def calculate_nearest(points, key_locations):
    distances = []
  
    key_union = key_locations.geometry.union_all()
    for point in points.geometry:
        nearest_geom = nearest_points(point, key_union)[1]
        distance = point.distance(nearest_geom)
        distances.append(distance)
    return distances

#  calculation for the nearest distance to schools, hospitals, and transportation hubs
NY_gdf['distance_to_school'] = calculate_nearest(NY_gdf, schools_gdf)
NY_gdf['distance_to_hospital'] = calculate_nearest(NY_gdf, hospitals_gdf)
NY_gdf['distance_to_transportation'] = calculate_nearest(NY_gdf, trans_gdf)


NY_gdf.head(3)

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,MAIN_ADDRESS,STATE,State_City,LATITUDE,...,bath_has_decimal,Location,Bike Score,Transit Score,Walk Score,Neighborhood,geometry,distance_to_school,distance_to_hospital,distance_to_transportation
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.0,1400.0,"2 E 55th St Unit 803New York, NY 10022","New York, NY 10022",New York,40.761255,...,False,"Midtown East, New York, 10022",82,100.0,100,"New York, NY 10022",POINT (-73.97448 40.76126),0.001492,0.011238,0.000699
1,Brokered by Serhant,Condo for sale,195000000,7,10.0,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",New York,40.766393,...,False,"Hell's Kitchen, New York, 10019",91,100.0,93,"New York, NY 10019",POINT (-73.98099 40.76639),0.003276,0.012035,0.000855
2,Brokered by COMPASS,Condo for sale,69000,3,1.0,445.0,"2 E 55th St Unit 908W33Manhattan, NY 10022","Manhattan, NY 10022",Manhattan,40.761398,...,False,"Midtown East, New York, 10022",82,100.0,100,"Manhattan, NY 10022",POINT (-73.97461 40.7614),0.001587,0.011326,0.000520


In [70]:
# converting degrees of longitude and latitude into meters
def degrees_to_meters(degrees, latitude, is_longitude=False):
    # Constants
    meters_per_degree_latitude = 111320

    if is_longitude:
        meters_per_degree_longitude = meters_per_degree_latitude * np.cos(np.radians(latitude))
        return degrees * meters_per_degree_longitude
    else:
        return degrees * meters_per_degree_latitude

# Convert distances
NY_gdf['dist_school_meters'] = degrees_to_meters(NY_gdf['distance_to_school'], NY_gdf['LATITUDE'], is_longitude=True)
NY_gdf['dist_hospital_meters'] = degrees_to_meters(NY_gdf['distance_to_hospital'] , NY_gdf['LATITUDE'], is_longitude=True)
NY_gdf['dist_transportation_meters'] = degrees_to_meters(NY_gdf['distance_to_transportation'] , NY_gdf['LATITUDE'], is_longitude=True)

NY_gdf.head(4)

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,MAIN_ADDRESS,STATE,State_City,LATITUDE,...,Transit Score,Walk Score,Neighborhood,geometry,distance_to_school,distance_to_hospital,distance_to_transportation,dist_school_meters,dist_hospital_meters,dist_transportation_meters
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.0,1400.000000,"2 E 55th St Unit 803New York, NY 10022","New York, NY 10022",New York,40.761255,...,100.0,100,"New York, NY 10022",POINT (-73.97448 40.76126),0.001492,0.011238,0.000699,125.837685,947.576865,58.954601
1,Brokered by Serhant,Condo for sale,195000000,7,10.0,17545.000000,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",New York,40.766393,...,100.0,93,"New York, NY 10019",POINT (-73.98099 40.76639),0.003276,0.012035,0.000855,276.243418,1014.645953,72.079494
2,Brokered by COMPASS,Condo for sale,69000,3,1.0,445.000000,"2 E 55th St Unit 908W33Manhattan, NY 10022","Manhattan, NY 10022",Manhattan,40.761398,...,100.0,100,"Manhattan, NY 10022",POINT (-73.97461 40.7614),0.001587,0.011326,0.000520,133.850000,955.012851,43.838720
3,Brokered by Douglas Elliman - 575 Madison Ave,Condo for sale,899500,2,2.0,2184.207862,"157 W 126th St Unit 1BNew York, NY 10027","New York, NY 10027",New York,40.809448,...,100.0,93,"New York, NY 10027",POINT (-73.94678 40.80945),0.001109,0.013684,0.002021,93.421984,1152.987304,170.252965


In [77]:
# spliting the location obtain by the website
NY_gdf[['Loc_Neighborhood', 'Loc_City', 'Loc_ZIP']] = NY_gdf['Location'].str.split(', ', expand=True)
NY_gdf.head(3)

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,MAIN_ADDRESS,STATE,State_City,LATITUDE,...,geometry,distance_to_school,distance_to_hospital,distance_to_transportation,dist_school_meters,dist_hospital_meters,dist_transportation_meters,Loc_Neighborhood,Loc_City,Loc_ZIP
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.0,1400.0,"2 E 55th St Unit 803New York, NY 10022","New York, NY 10022",New York,40.761255,...,POINT (-73.97448 40.76126),0.001492,0.011238,0.000699,125.837685,947.576865,58.954601,Midtown East,New York,10022
1,Brokered by Serhant,Condo for sale,195000000,7,10.0,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",New York,40.766393,...,POINT (-73.98099 40.76639),0.003276,0.012035,0.000855,276.243418,1014.645953,72.079494,Hell's Kitchen,New York,10019
2,Brokered by COMPASS,Condo for sale,69000,3,1.0,445.0,"2 E 55th St Unit 908W33Manhattan, NY 10022","Manhattan, NY 10022",Manhattan,40.761398,...,POINT (-73.97461 40.7614),0.001587,0.011326,0.000520,133.850000,955.012851,43.838720,Midtown East,New York,10022


In [74]:
NY_gdf.isna().sum()

BROKERTITLE                   0
TYPE                          0
PRICE                         0
BEDS                          0
BATH                          0
PROPERTYSQFT                  0
MAIN_ADDRESS                  0
STATE                         0
State_City                    0
LATITUDE                      0
LONGITUDE                     0
beds_has_decimal              0
bath_has_decimal              0
Location                      0
Bike Score                    0
Transit Score                 6
Walk Score                    0
Neighborhood                  0
geometry                      0
distance_to_school            0
distance_to_hospital          0
distance_to_transportation    0
dist_school_meters            0
dist_hospital_meters          0
dist_transportation_meters    0
Loc_Neighborhood              0
Loc_City                      0
Loc_ZIP                       7
dtype: int64

In [78]:
NY_gdf.shape

(1182, 28)

In [80]:
DATASET = NY_gdf[['BROKERTITLE','TYPE','STATE','State_City','Neighborhood','Location', 'Loc_Neighborhood','Loc_City','Loc_ZIP',
                  'LATITUDE','LONGITUDE','geometry','PRICE','BEDS','BATH','PROPERTYSQFT','Walk Score',
                  'Transit Score','Bike Score','dist_transportation_meters','dist_hospital_meters','distance_to_school']]

In [81]:
DATASET.head()

,BROKERTITLE,TYPE,STATE,State_City,Neighborhood,Location,Loc_Neighborhood,Loc_City,Loc_ZIP,LATITUDE,...,PRICE,BEDS,BATH,PROPERTYSQFT,Walk Score,Transit Score,Bike Score,dist_transportation_meters,dist_hospital_meters,distance_to_school
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,"New York, NY 10022",New York,"New York, NY 10022","Midtown East, New York, 10022",Midtown East,New York,10022,40.761255,...,315000,2,2.0,1400.000000,100,100.0,82,58.954601,947.576865,0.001492
1,Brokered by Serhant,Condo for sale,"New York, NY 10019",New York,"New York, NY 10019","Hell's Kitchen, New York, 10019",Hell's Kitchen,New York,10019,40.766393,...,195000000,7,10.0,17545.000000,93,100.0,91,72.079494,1014.645953,0.003276
2,Brokered by COMPASS,Condo for sale,"Manhattan, NY 10022",Manhattan,"Manhattan, NY 10022","Midtown East, New York, 10022",Midtown East,New York,10022,40.761398,...,69000,3,1.0,445.000000,100,100.0,82,43.838720,955.012851,0.001587
3,Brokered by Douglas Elliman - 575 Madison Ave,Condo for sale,"New York, NY 10027",New York,"New York, NY 10027","Manhattan, New York, 10027",Manhattan,New York,10027,40.809448,...,899500,2,2.0,2184.207862,93,100.0,60,170.252965,1152.987304,0.001109
4,Brokered by Corcoran Chelsea/Flatiron,Co-op for sale,"New York, NY 10014",New York,"New York, NY 10014","West Village, New York, 10014",West Village,New York,10014,40.738298,...,895000,3,1.0,2184.207862,98,100.0,94,54.583331,428.760776,0.004152


In [82]:
# convert to csv and save
file_path = r'C:/Users/1999i/Documents/data mange/Lab Lectures/Project/Data/DATASET_NY_COUNTY.csv'
DATASET.to_csv(file_path, index=False)